In [1]:
from selenium import webdriver

In [2]:
# open chrome browser
browser = webdriver.Chrome(executable_path='../../../../chromedriver.exe')

In [3]:
browser.get('https://play.google.com/store/search?q=%ED%97%AC%EC%8A%A4%EC%BC%80%EC%96%B4%EC%95%B1&c=apps&hl=ko-KR')

In [4]:
browser.implicitly_wait(10)

In [5]:
import pandas as pd
import pymongo as mg
import time

In [6]:
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['study_data_analytics']
collection = database['googlestore_healthcare']


# 서비스 앱에 대한 리스트
click_path = 'section > div > div > div > div'
company_list = browser.find_elements_by_css_selector(click_path)

len(company_list)

for company in company_list:
    company.click()
    time.sleep(2)

    # 서비스 앱에 대한 정보 가져오기
    try:
        service_name = browser.find_element_by_css_selector('div > h1 > span').text
    except :
        service_name = str()
    try :    
        company_name = browser.find_element_by_css_selector('div.Il7kR > div > div > div > a > span').text
    except : 
        company_name = str()
    try :    
        app_information = browser.find_element_by_css_selector('div:nth-child(1) > c-wiz:nth-child(2) > div > section > div').text
    except :
        app_information = str()
    #### 리뷰 총 갯수 가져오기
    ## 리뷰 총 갯수 tag : div:nth-child(1) > div.g1rdde
    review_total_count_text = browser.find_element_by_css_selector('div:nth-child(1) > div.g1rdde').text
    
    import re  # regexp function
    result_list = re.findall(r'\d+', review_total_count_text)
    print(result_list[0], int(result_list[0]))

    review_total_count = int(result_list[0])  # 리뷰 총 갯수    

    # [service_name, company_name, app_information]

    # 댓글 팝업 띄우기(해당 리뷰 없을 시 대비한 방어 코드 작성 필요)
    review_flag = True
    try :
        browser.find_element_by_xpath('//*/div/div/button/*[contains(text(), "리뷰")]').click()    
    except :
        review_flag = False
    
    # 회사 서비스에 대한 전체 리뷰 가져오기
    serviceapp_columns_name = ['service_name', 'content', 'date', 'stars']
    reviews_list = list()
    if review_flag :
        # 팝업 띄우기
        reviews_bundle = browser.find_elements_by_css_selector('div.fysCi > div > div:nth-child(2) > div')
        len(reviews_bundle)    

        # 팝업 속 댓글 전체표시
        review_standard_count_per = 40

        loop_count_int = int(review_total_count / review_standard_count_per)
        for count in range(1, loop_count_int+1) :
            try :
                reviews_bundle = browser.find_elements_by_css_selector('div.fysCi > div > div:nth-child(2) > div')
                print('current reviews_bundle count : {}'.format(len(reviews_bundle)))
                reviews_bundle[len(reviews_bundle)-1].click()
                # print('current reviews_bundle count : {}'.format(len(reviews_bundle)))
                time.sleep(3)
            except :
                print('pass')
                pass
        print('Done', len(reviews_bundle))
        
        # 팝업 댓글 수집
        for review_bundle in reviews_bundle:
            content = review_bundle.find_element_by_css_selector('div.h3YV2d').text
            date = review_bundle.find_element_by_css_selector('header > div.Jx4nYe > span').text
            stars = review_bundle.find_element_by_css_selector('header > div.Jx4nYe > div').get_attribute('aria-label')
            review_list = [service_name, content, date, stars]
            reviews_list.append(review_list)  # 리뷰에 대한 모든 것(특정 회사 서비스에 대한)

        browser.back() # 팝업 닫기
        # 저장 위한 작업
        df_reviews = pd.DataFrame(data=reviews_list, columns=serviceapp_columns_name)
        data_dict = df_reviews.to_dict(orient='records')
        collection.insert_many(data_dict)
        
    print('service_name : {}, review count : {}'.format(service_name, len(reviews_list)))
    browser.back() # 리스트로 돌아가기
    time.sleep(3)
print('Done')

145 145
current reviews_bundle count : 40
current reviews_bundle count : 80
current reviews_bundle count : 120
Done 120
service_name : Samsung Health(삼성 헬스), review count : 120
473 473
service_name : NH헬스케어, review count : 0
30 30
service_name : 케어헬스, review count : 0
9 9
Done 7
service_name : 기업용 디지털 헬스케어 삼성화재 애니핏 프로(PRO), review count : 7
3 3
Done 40
service_name : 애니핏 플러스 나에게 핏한 건강관리, review count : 40
7 7
service_name : SK VIEW 헬스케어, review count : 0
33 33
Done 14
service_name : 헬스케치-건강검진 기반 건강지표/위험도 분석,맞춤 헬스, review count : 14
6 6
Done 1
service_name : 헬스케어 건강관리서비스, review count : 1
30 30
service_name : PT쌤 - 운동 헬스 체중관리 체력테스트 홈트 기록, review count : 0
3 3
service_name : 플릭 - 운동일지, 운동기록, 운동일기, 운동루틴, review count : 0
145 145
current reviews_bundle count : 40
current reviews_bundle count : 71
current reviews_bundle count : 71
Done 71
service_name : Health4U, review count : 71
1 1
Done 40
service_name : 플랜핏 - 헬스 홈트 운동 루틴 추천과 피트니스 기록, review count : 40
274 274
current reviews_bundle coun

IndexError: list index out of range

In [ ]:
reviews_list